<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="./assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

An equity trading simulation, with 4 Traders and a Researcher, powered by a 6 MCP servers with 44 tools & 2 resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data from Polygon
6. The return of our Push Notification tool ❤️

And a resource to read information about the trader's account, and their investment strategy.

This lab looks at a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="./assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [ ]:
pip install dotenv mcp openai-agents polygon-api-client

In [2]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP servers for our trader and researcher

In [3]:

from market import is_paid_polygon, is_realtime_polygon

load_dotenv(override=True)

brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
polygon_api_key = os.getenv("POLYGON_API_KEY")

# The MCP server for the Trader to read Market Data

if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})


# The full set of MCP servers for the trader: Accounts, Push Notification and the Market

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

def researcher_mcp_server_params(name: str):
    return [
        {"command": "uvx", "args": ["mcp-server-fetch"]},
        {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}},
        {"command": "npx", "args": ["-y", "mcp-memory-libsql"], "env": {"LIBSQL_URL": f"file:{name}.db"}}
    ]
trader_mcp_servers = [MCPServerStdio(params) for params in trader_mcp_server_params]
researcher_mcp_servers = [MCPServerStdio(params) for params in researcher_mcp_server_params("amar")]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool

In [4]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )
    return researcher


async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [4]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    print("Researcher is ready.", researcher)
    result = await Runner.run(researcher, research_question)
display(Markdown(result.final_output))


Researcher is ready. Agent(name='Researcher', instructions="You are a financial researcher. You are able to search the web for interesting financial news,\nlook for possible trading opportunities, and help with research.\nBased on the request, you carry out necessary research and respond with your findings.\nTake time to make multiple searches to get a comprehensive overview, and then summarize your findings.\nIf there isn't a specific request, then just respond with investment opportunities based on searching latest news.\nThe current datetime is 2025-05-20 14:27:08\n", handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[<agents.mcp.server.MCPServerStdio ob

Here are the latest updates on Amazon:

1. **Financial Performance**:
   - Amazon reported its first-quarter financial results for 2025, showing a 9% increase in net sales to **$155.7 billion**, compared to **$143.3 billion** in the same quarter last year. This growth reflects strong operational performance despite market challenges. More details can be found on [Amazon's Investor Relations](https://ir.aboutamazon.com/news-release/news-release-details/2025/Amazon-com-Announces-First-Quarter-Results/).

2. **Stock Information**:
   - The stock quote for Amazon (AMZN) can be tracked on financial platforms like Yahoo Finance, where you can find the latest price, history, and news relevant to the stock. Check it out [here](https://finance.yahoo.com/quote/AMZN/).

3. **Market Impact**:
   - There were discussions around Amazon's business strategies and stock performance, particularly following recent economic trends that might affect investor sentiment. Coverage includes various analytical reports available on platforms such as CBS News and Markets Insider.

4. **Operational Insights**:
   - As part of its growth strategy, Amazon continues to focus on customer obsession, innovation, and operational excellence. This approach is expected to bolster its long-term market position.

5. **Company Announcements**:
   - Amazon has also been involved in various public engagements and reports, further solidifying its brand image and market presence. More details can be followed on their dedicated news channels.

For ongoing updates and detailed overviews, you might want to explore comprehensive financial news platforms such as [Global News](https://globalnews.ca/tag/amazon/) and [CBS News](https://www.cbsnews.com/tag/amazon/).

### Look at the trace

https://platform.openai.com/traces

In [8]:
from mcp import StdioServerParameters

amar_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Amar").reset(amar_initial_strategy)
params = StdioServerParameters(command="uv", args=["run", "accounts_server.py"], env=None)
account_server = MCPServerStdio({"command": "uv", "args": ["run", "accounts_server.py"]})

async def read_accounts_resource(name, server) -> str:
    """Read account information for the given name.
    
    Args:
        name: The name of the account holder
        
    Returns:
        Account report as a string
    """
    print(f"Reading account resource for {name}")
    
    
    result = await account_server.session.read_resource(f"accounts://accounts_server/{name}")
    print("Result: ", result)
    return result.contents[0].text
    # async with stdio_client(params) as streams:
    #     print("params" , params)
    #     print("streams", streams)
    #     async with mcp.ClientSession(*streams) as session:
    #         print("Session created", session)    
    #         print("Session initialized")
    #         result = await session.read_resource(f"accounts://accounts_server/{name}")
    #         print("result", result)
    #         return result.contents[0].text
    

await account_server.connect()
display(Markdown(await read_accounts_resource(name="Amar", server=account_server)))

Reading account resource for Amar
Result:  meta=None contents=[TextResourceContents(uri=AnyUrl('accounts://accounts_server/Amar'), mimeType='text/plain', text='{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 14:55:00", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}')]


{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 14:55:00", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

In [10]:

display(Markdown(await read_accounts_resource("Amar", account_server)))
display(Markdown(await read_strategy_resource("Amar")))

Reading account resource for Amar
Result:  meta=None contents=[TextResourceContents(uri=AnyUrl('accounts://accounts_server/Amar'), mimeType='text/plain', text='{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 14:55:00", 10000.0], ["2025-05-20 14:56:32", 10000.0], ["2025-05-20 14:56:40", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}')]


{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 14:55:00", 10000.0], ["2025-05-20 14:56:32", 10000.0], ["2025-05-20 14:56:40", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [12]:
agent_name = "Amar"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name, account_server)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

Reading account resource for Amar
Result:  meta=None contents=[TextResourceContents(uri=AnyUrl('accounts://accounts_server/Amar'), mimeType='text/plain', text='{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 14:55:00", 10000.0], ["2025-05-20 14:56:32", 10000.0], ["2025-05-20 14:56:40", 10000.0], ["2025-05-20 14:57:10", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}')]


In [13]:
display(Markdown(instructions))


You are a trader that manages a portfolio of shares. Your name is Amar and your account is under your name, Amar.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "amar", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-05-20 14:55:00", 10000.0], ["2025-05-20 14:56:32", 10000.0], ["2025-05-20 14:56:40", 10000.0], ["2025-05-20 14:57:10", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.


### And to run our Trader

In [14]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt)
display(Markdown(result.final_output))

### Summary of Trades Executed

1. **Buy AMZN (Amazon)**
   - **Shares Bought**: 25
   - **Price**: $34.068 per share
   - **Rationale**: Analysts suggest this may be the last chance to buy Amazon under $200 with good future gains anticipated.
   - **Transaction Timestamp**: 2025-05-20 14:59:18

2. **Buy NVDA (Nvidia)**
   - **Shares Bought**: 15
   - **Price**: $91.182 per share
   - **Rationale**: Nvidia's advancements in technology and positive market sentiment make it a strong investment opportunity.
   - **Transaction Timestamp**: 2025-05-20 14:59:18

### Portfolio Update
- **Current Balance**: $7780.57
- **Holdings**:
  - Amazon: 25 shares
  - Nvidia: 15 shares
- **Total Portfolio Value**: $10,895.57
- **Total Profit/Loss**: +$895.57

### Conclusion
These trades position the portfolio well within the booming tech sector, especially focusing on companies at the forefront of the AI revolution. Monitoring of these positions will continue as the market evolves.

### Now it's time to review the Python module made from this:

`mcp_servers.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.
